## MNIST – MLP solution

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Input, Flatten
from keras.backend import clear_session

# target binary encoding
from keras.utils import to_categorical
# from sklearn.preprocessing import LabelBinarizer  

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report

import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_dark'

Load MNIST dataset

In [2]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(f"single sample shape:   {X_train[0].shape}")
print(f"samples in train data: {X_train.shape[0]}")
print(f"samples in test data:  {X_test.shape[0]}")

single sample shape:   (28, 28)
samples in train data: 60000
samples in test data:  10000


In [3]:
print(*y_train[:7])

px.imshow(np.hstack(X_train[:7]), width=1000, height=300).update_coloraxes(showscale=False)

5 0 4 1 9 2 1


Flatten samples to 1D-arrays

In [4]:
X_train_flat = np.reshape(X_train, (X_train.shape[0], -1))
X_test_flat = np.reshape(X_test, (X_test.shape[0], -1))
X_train_flat.shape, X_test_flat.shape

((60000, 784), (10000, 784))

Scale (normalize) data

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)

Target binary encoding

In [6]:
y_train_bnr = to_categorical(y_train)
y_test_bnr = to_categorical(y_test)

Model assembling

In [7]:
model = Sequential([
    Input((X_train_scaled.shape[1],)),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(y_train_bnr.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,282 (462.04 KB)

 Trainable params: 118,282 (462.04 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
history = model.fit(
    X_train_scaled, y_train_bnr, 
    epochs=25, 
    batch_size=64,
    validation_data=(X_test_scaled, y_test_bnr)
)

Epoch 1/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8762 - loss: 0.4143 - val_accuracy: 0.9608 - val_loss: 0.1305
Epoch 2/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9705 - loss: 0.0959 - val_accuracy: 0.9674 - val_loss: 0.1193
Epoch 3/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9809 - loss: 0.0631 - val_accuracy: 0.9708 - val_loss: 0.1163
Epoch 4/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9865 - loss: 0.0435 - val_accuracy: 0.9706 - val_loss: 0.1247
Epoch 5/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9887 - loss: 0.0337 - val_accuracy: 0.9730 - val_loss: 0.1391
Epoch 6/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9913 - loss: 0.0267 - val_accuracy: 0.9721 - val_loss: 0.1320
Epoch 7/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9915 - loss: 0.0263 - val_accuracy: 0.9749 - val_loss: 0.1318
Epoch 8/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9937 - loss: 0.0227 - val_accuracy: 0.

In [11]:
fig_data = pd.DataFrame({
    'accuracy train': history.history['accuracy'],
    'accuracy val': history.history['val_accuracy'],
    'loss train': history.history['loss'],
    'loss val': history.history['val_loss']
})

px.line(
    fig_data,
    y=['loss train', 'loss val'],
    width=600, height=400
). show()

px.line(
    fig_data,
    y=['accuracy train', 'accuracy val'],
    width=600, height=400
). show()

In [10]:
y_pred = np.argmax(model.predict(X_test_scaled), axis=1)

print(classification_report(y_test, y_pred))

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.96      0.98      0.97      1032
           3       0.99      0.95      0.97      1010
           4       0.97      0.98      0.98       982
           5       0.96      0.98      0.97       892
           6       0.98      0.97      0.98       958
           7       0.97      0.97      0.97      1028
           8       0.97      0.97      0.97       974
           9       0.97      0.97      0.97      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000

